In [5]:
from Plan import CreateSamples_renewed
from Prepare import OT2Commands as ALH
from Prepare import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api
import importlib

In [12]:
importlib.reload(CreateSamples_renewed)
importlib.reload(ALH)

<module 'Prepare.OT2Commands' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\PlanPrepareProcess\\Prepare\\OT2Commands.py'>

In [14]:
path = r"Testing Plans\04_28_21_CdSe_Ligand_Study.csv"
chem_path = r"Chemical Database.csv"
plan = CreateSamples_renewed.get_experiment_plan(path, chem_path)
plan

{'Sample Amount': 5,
 'Sample Unit': 'mL',
 'Component Shorthand Names': ['CdAc', 'Se', 'OleicAc', 'Oleylamine'],
 'Component Concentration Units': ['molarity',
  'molarity',
  'molarity',
  'molarity'],
 'Component Concentration Linspaces [min, max, n]': [[0.06286, 0.06286, 1],
  [0.25167, 0.25167, 1],
  [0.258, 0.258, 1],
  [0.124, 0.124, 1]],
 'Stock Names': ['CdAc-TOP-stock',
  'Se-TOP-stock',
  'OleicAc-stock',
  'Oleylamine-stock'],
 'Stock Concentration Units': ['molarity', 'molarity', 'volf', 'volf'],
 'Stock Concentrations': [0.5, 1, 1, 1],
 'OT2 Destination Labwares': ['falcon_48_wellplate_1500ul',
  'falcon_48_wellplate_1500ul',
  'falcon_48_wellplate_1500ul'],
 'OT2 Destination Labware Slots': ['2', '3', '5'],
 'OT2 Sample Labware Start': 0,
 'OT2 Stock Labwares': ['20mlscintillation_12_wellplate_18000ul',
  '20mlscintillation_12_wellplate_18000ul'],
 'OT2 Stock Labware Slots': ['1', '4'],
 'OT2 Stock Labware Stock Order': [1, 2, 3, 4, 5],
 'OT2 Left Pipette': 'p300_single_

In [15]:
concentration_df = CreateSamples_renewed.concentration_from_linspace(plan['Component Shorthand Names'], 
                                                                     plan['Component Concentration Linspaces [min, max, n]'],
                                                                     plan['Component Concentration Units'],
                                                                     unity_filter=False)
concentration_df

,CdAc concentration molarity,Se concentration molarity,OleicAc concentration molarity,Oleylamine concentration molarity
0,0.06286,0.25167,0.258,0.124


In [214]:
complete_amounts_added = CreateSamples_renewed.determine_component_amounts(plan, concentration_df, nan_fill_value=0)
stock_dict = CreateSamples_renewed.stock_dictionary(plan['Stock Names'], plan['Stock Concentration Units'], plan['Stock Concentrations'], plan['Stock Density (g/mL) (only for wtf)']) # make it so when creating a stock_dict that if missing anything just raise an but still allow and fill missing with nan
complete_df = CreateSamples_renewed.calculate_stock_volumes_from_component_masses(plan, complete_amounts_added, stock_dict)
complete_df = CreateSamples_renewed.calculate_common_solvent_residual_volumes(complete_df, stock_dict)

You calculated for component masses given the provided units
You calculated for component masses given the provided units


In [215]:
stock_volumes = CreateSamples_renewed.isolate_common_column(complete_df,'stock')

In [216]:
stock_volumes = CreateSamples_renewed.complete_missing_volume_with_commmon_solvent(plan['Sample Amount'], stock_volumes, stock_dict, 'water')

In [218]:
CreateSamples_renewed.convert_mL_to_uL(complete_df)

,Y(NO3)3 amount volume uL,KBF4 amount volume uL,Y(NO3)3-water-stock amount volume uL,KBF4-water-stock amount volume uL
0,10.71057,7.538922,75.0,4213.483146


In [199]:
labware_dir_path = r"Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)
protocol = simulate.get_protocol_api('2.3', extra_labware=custom_labware_dict) # make note this must be run constantly ~ if say making loaded dict twice

C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults


In [200]:
loaded_dict = ALH.loading_labware(protocol, plan)
loaded_dict = ALH.determine_pipette_resolution(loaded_dict) # this should just be automatically called move into function

In [201]:
max_source_vol = 15000
stock_position_info = ALH.stock_well_ranges(stock_volumes, loaded_dict, max_source_vol) 
stock_position_info

{'Y(NO3)3-water-stock amount volume mL': {'Ranges': [[0, 1]],
  'Stock Wells': [A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'KBF4-water-stock amount volume mL': {'Ranges': [[0, 1]],
  'Stock Wells': [A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]},
 'water-stock volume mL': {'Ranges': [[0, 1]],
  'Stock Wells': [A3 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1]}}

In [202]:
directions = ALH.create_sample_making_directions(stock_volumes, stock_position_info, loaded_dict)
directions

AssertionError: Sample volumes are exceeding max destination well volume of 1500.0 uL

In [178]:
transfer_info = ALH.pipette_volumes_sample_wise(protocol, directions, loaded_dict)

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 30.000042611565522 from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A5 of Falcon 48 Well Plate 1500 ÂµL on 2
Aspirating 30.000042611565522 uL from A1 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 30.000042611565522 uL into A5 of Falcon 48 Well Plate 1500 ÂµL on 2 at 300.0 uL/sec
Returning tip
Dropping tip into A1 of Opentrons 96 Tip Rack 300 µL on 10
Picking up tip from A2 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 33.44656189053347 from A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A5 of Falcon 48 Well Plate 1500 ÂµL on 2
Aspirating 33.44656189053347 uL from A2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 50.0 uL/sec
Dispensing 33.44656189053347 uL into A5 of Falcon 48 Well Plate 1500 ÂµL on 2 at 300.0 uL/sec
Returning tip
Dropping tip into A2 of Opentrons 96 Tip Rack 300 µL on 10
Picking up tip from A3 of Opentrons 96 Tip Rack 300 µL on 10
Tran

In [ ]:
directions[0].keys()